## Model Layers
该模块包含许多我们可能有兴趣在模型中使用的图层类。 这些图层补充了我们也可以用作预定义图层的默认Pytorch图层。

### Custom fastai modules
<b>class AdaptiveConcatPool2d</b>

`AdaptiveConcatPool2d(sz:Optional[int]=None) :: Module`

```python
class AdaptiveConcatPool2d(nn.Module):
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`."
    def __init__(self, sz:Optional[int]=None):
        "Output will be 2*sz or 2 if sz is None"
        super().__init__()
        self.output_size = sz or 1
        self.ap = nn.AdaptiveAvgPool2d(self.output_size)
        self.mp = nn.AdaptiveMaxPool2d(self.output_size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
```

连接AdaptiveAvgPool2d和AdaptiveMaxPool2d的层。

输出为2 * sz，如果sz为None，则输出为2。

AdaptiveConcatPool2d对象使用自适应平均池和自适应最大池并将它们连接起来。 我们使用它是因为它为模型提供了两种方法的信息并提高了性能。 这种技术称为自适应，因为它允许我们决定我们想要的输出尺寸，而不是选择输入的尺寸以适合所需的输出尺寸。

让我们首先尝试使用自适应平均池进行训练，然后使用自适应最大池进行训练，最后使用它们的串联来查看它们在性能方面的表现。

我们将首先通过稍微更改源代码来使用Adaptive Max Pooling定义simple_cnn。

In [1]:
from fastai.vision import *
from fastai.basics import *
from fastai.callbacks import *

In [2]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [3]:
def simple_cnn_max(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveMaxPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [4]:
model = simple_cnn_max((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.119670,0.082491,0.978901,00:57


现在让我们尝试使用Adaptive Average Pooling。

In [5]:
def simple_cnn_avg(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [6]:
model = simple_cnn_avg((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.289376,0.248056,0.974485,00:57


最后，我们将尝试使用AdaptiveConcatPool2d将它们连接起来。 事实上，我们会看到它提高了我们的准确性并大大减少了我们的损失！

In [7]:
def simple_cnn(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(AdaptiveConcatPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [8]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.142278,0.091850,0.982336,00:56


<b>class Lambda</b>

`Lambda(func:LambdaFunc) :: Module`

```python
class Lambda(nn.Module):
    "An easy way to create a pytorch layer for a simple `func`."
    def __init__(self, func:LambdaFunc):
        "create a layer that simply calls `func` with `x`"
        super().__init__()
        self.func=func

    def forward(self, x): return self.func(x)
```
为简单的func创建pytorch层的简单方法。

这对于在Sequential对象内部的网络中使用函数非常有用。 因此，例如，假设我们想要应用log_softmax损失，我们需要更改输出批次的形状以便能够使用此损失。 我们可以通过调用添加一个应用必要的形状变化的图层：

Lambda（lambda x：x.view（x.size（0）， -  1））

让我们看一个示例，说明添加此图层时输出的形状如何变化。

In [9]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10, 1, 1])


In [10]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0),-1))
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


<b>class Flatten</b>

`Flatten(full:bool=False) :: Module`
        
```python
class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full:bool=False):
        super().__init__()
        self.full = full

    def forward(self, x):
        return x.view(-1) if self.full else x.view(x.size(0), -1)

```
将x展平为单个维度，通常在模型的末尾使用。 完全为1级张量

我们上面构建的函数实际上在我们的库中实现为Flatten。 我们可以看到它在运行时返回相同的大小。


In [12]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


<b>PoolFlatten</b>

`PoolFlatten() → Sequential`

```python

```
将nn.AdaptiveAvgPool2d应用于x，然后展平结果。

我们可以使用PoolFlatten将这两个最终层（AdaptiveAvgPool2d和Flatten）组合在一起。

In [14]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    PoolFlatten()
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


我们给Lambda函数的另一个用途是当我们有一个期望输入不同于前一个的层时，使用ResizeBatch调整批量大小。

<b>class ResizeBatch</b>

`ResizeBatch(*size:int) :: Module`
        
重塑x到大小，保持批量调暗相同的大小

In [16]:
a = torch.tensor([[1., -1.], [1., -1.]])[None]
print(a)

tensor([[[ 1., -1.],
         [ 1., -1.]]])


In [17]:
out = ResizeBatch(4)
print(out(a))

tensor([[ 1., -1.,  1., -1.]])


<b>class Debugger</b>

`Debugger() :: Module`

用于在模型内部进行调试的模块。

调试器模块允许我们在培训期间窥视网络，并详细了解正在发生的事情。 我们可以在网络中的任何位置查看输入，输出和大小。

例如，如果您运行以下内容：

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    Debugger(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
)

model.cuda()通过添加或连接密集= True的主题，将快捷方式与模块的结果合并。

epoch,train_loss,valid_loss,accuracy,time


> c:\users\yang\appdata\local\conda\conda\envs\pytorch\lib\site-packages\fastai\layers.py(219)forward()
-> return x


<b>class PixelShuffle_ICNR</b>

`PixelShuffle_ICNR(ni:int, nf:int=None, scale:int=2, blur:bool=False, norm_type=<NormType.Weight: 3>, leaky:float=None) :: Module`

使用nn.PixelShuffle，`icnr` init和`weight_norm`按比例从`ni`过滤器到`nf`（默认`ni`）。

<b>class MergeLayer</b>

`MergeLayer(dense:bool=False) :: Module`

通过添加或连接`dense=True`的主题，将快捷方式与模块的结果合并。

<b>class PartialLayer</b>

`PartialLayer(func, **kwargs) :: Module`

应用部分（func，** kwargs）的图层。

<b>class SigmoidRange</b>

`SigmoidRange(low, high) :: Module`

具有范围（低，x_max）的Sigmoid模块

<b>class SequentialEx</b>

`SequentialEx(*layers) :: Module`

像nn.Sequential，但具有ModuleList语义，并且可以访问模块输入

<b>class SelfAttention</b>

`SelfAttention(n_channels:int) :: Module`

自我关注层为nd。

<b>class BatchNorm1dFlat</b>

`BatchNorm1dFlat(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True) :: BatchNorm1d`

nn.BatchNorm1d，但首先展平领先尺寸

## Loss functions
<b>class FlattenedLoss</b>

`FlattenedLoss(func, *args, axis:int=-1, floatify:bool=False, is_2d:bool=True, **kwargs)`

与func相同，但会使输入和目标变平。

使用args和kwargs创建一个func实例。 传递输出和目标时，它

* 首先将轴放在输出中并使用转置设置目标
* 将目标转换为float是floatify = True
* 如果为_2d，则将输出压缩为二维，否则将一个维度挤压到一维
* 应用了func的实例。

<b>BCEFlat</b>

`BCEFlat(*args, axis:int=-1, floatify:bool=True, **kwargs)`

与nn.BCELoss相同，但输入和目标变平。

<b>BCEWithLogitsFlat</b>

`BCEWithLogitsFlat(*args, axis:int=-1, floatify:bool=True, **kwargs)`

与nn.BCEWithLogitsLoss相同，但会使输入和目标变平。

<b>CrossEntropyFlat</b>

`CrossEntropyFlat(*args, axis:int=-1, **kwargs)`

与nn.CrossEntropyLoss相同，但会使输入和目标变平。

<b>MSELossFlat</b>

`MSELossFlat(*args, axis:int=-1, floatify:bool=True, **kwargs)`

与nn.MSELoss相同，但会使输入和目标变平。

## class NoopLoss

`NoopLoss() :: Module`

只返回输出的平均值。

### class WassersteinLoss

`WassersteinLoss() :: Module`

For WGAN.

## Helper functions to create modules
<b>bn_drop_lin</b>

`bn_drop_lin(n_in:int, n_out:int, bn:bool=True, p:float=0.0, actn:Optional[Module]=None)`

bn_drop_lin函数返回批量标准化，dropout和线性层的序列。 此自定义图层通常在模型的末尾使用。

n_in表示输入大小的数量n_out输出的大小，bn我们是否需要批处理规范，p是多少dropout和actn是一个可选参数，用于在最后添加激活函数。

<b>conv2d</b>

`conv2d(ni:int, nf:int, ks:int=3, stride:int=1, padding:int=None, bias=False, init:LayerFunc='kaiming_normal_') → Conv2d`

创建并初始化nn.Conv2d图层。 padding默认为ks // 2。

<b>conv2d_trans</b>

`conv2d_trans(ni:int, nf:int, ks:int=2, stride:int=2, padding:int=0, bias=False) → ConvTranspose2d`

创建nn.ConvTranspose2d图层。

<b>conv_layer</b>

`conv_layer(ni:int, nf:int, ks:int=3, stride:int=1, padding:int=None, bias:bool=None, is_1d:bool=False, norm_type:Optional[NormType]=<NormType.Batch: 1>, use_activ:bool=True, leaky:float=None, transpose:bool=False, init:Callable='kaiming_normal_', self_attention:bool=False)`

conv_layer函数返回nn.Conv2D，BatchNorm和ReLU或泄漏RELU激活函数的序列。

n_in表示输入大小的数量n_out输出大小，ks内核大小，跨越我们想要应用卷积的步幅。 偏见将决定他们是否有偏见（如果是None，默认为True，除非使用batchnorm）。 norm_type选择规范化类型（或无）。 如果泄漏是无，则激活是标准ReLU，否则它是斜率泄漏的LeakyReLU。 最后，如果transpose = True，则卷积由ConvTranspose2D替换。

<b>embedding</b>

`embedding(ni:int, nf:int) → Module`

创建一个输入大小为ni且输出大小为nf的嵌入层

<b>relu</b>

`relu(inplace:bool=False, leaky:float=None)`

返回relu激活，可能是`leaky` 和 `nplace`.

<b>res_block</b>

`res_block(nf, dense:bool=False, norm_type:Optional[NormType]=<NormType.Batch: 1>, bottle:bool=False, **conv_kwargs)`

Resnet块的nf功能。 conv_kwargs传递给conv_layer。

<b>sigmoid_range</b>

`sigmoid_range(x, low, high)`

具有范围（低，高）的Sigmoid函数

<b>simple_cnn</b>

`simple_cnn(actns:Collection[int], kernel_szs:Collection[int]=None, strides:Collection[int]=None, bn=False) → Sequential`

CNN，其中conv_layer由actns，kernel_szs和strides定义，加上batchnorm如果bn。

## Initialization of modules
<b>batchnorm_2d</b>

`batchnorm_2d(nf:int, norm_type:NormType=<NormType.Batch: 1>)`

具有nf特征的batchnorm2d层根据norm_type初始化。

<b>icnr</b>

`icnr(x, scale=2, init='kaiming_normal_')`

ICNR init of x，具有scale和init功能。

<b>trunc_normal_</b>

`trunc_normal_(x:Tensor, mean:float=0.0, std:float=1.0) → Tensor`

截断正常初始化。

<b>icnr</b>

`icnr(x, scale=2, init='kaiming_normal_')`

ICNR init of x，具有scale和init功能。

## NormType
`Enum = [Batch, BatchZero, Weight, Spectral]`

枚举。